Copyright **`(c)`** 2023 Nicolò Caradonna `<s316993@studenti.polito.it>`  
[`https://github.com/Nicocarad/Computational-Intelligence`] 
Worked with Angelo Iannielli - s317887

In [855]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue
from math import ceil
from tqdm.auto import tqdm

import numpy as np

In [856]:
PROBLEM_SIZE = 15
NUM_SETS = 25
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [ ]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

In [857]:

    
def g(state):
    return len(state.taken)



# number of positions to cover to reach the goal  
def h1(state):
    return PROBLEM_SIZE - sum(
        covered(state))


def h2(state):
    covered_tiles = sum(covered(state))
    if covered_tiles == PROBLEM_SIZE:
        return 0
    return 1 / covered_tiles if covered_tiles != 0 else 1
    

# We only considered the sets not taken, 
# so as not to be influenced by the existence of large sets which have already been taken
def h3(state):
    not_taken_sets = [s for i, s in enumerate(SETS) if i not in state.taken]
    largest_set_size = max(sum(s) for s in not_taken_sets) # select the larget tiles (more number of true)
    missing_size = PROBLEM_SIZE - sum(covered(state)) # evaluates the number of tiles that are not covered
    optimistic_estimate = ceil(missing_size / largest_set_size) # estimate the number of set that are missing for the solution in a optimistic way
    # if the largest set is 5 and the missing size is 10 --> "maybe" 2 sets are missing (optimistic assumption)
    return optimistic_estimate


def f1(state):
    cost_1 = g(state)
    cost_2 = h1(state)
    
    return cost_1 + cost_2
    
# since h2 is a value between 0 and 1, we multiply it by 0.1 to make it more significant 
def f2(state):
    cost_1 = 0.1*g(state)
    cost_2 = h2(state)
    
    return cost_1 + cost_2
    
def f3(state):
    cost_1 = g(state)
    cost_2 = h3(state)
    
    return cost_1 + cost_2





In [858]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

# Solution with h1

In [859]:

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f1(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f1(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 3 steps (3 tiles)


In [860]:
current_state

State(taken={2, 12, 6}, not_taken={0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24})

# Solution with h2

In [861]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f2(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f2(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 47 steps (3 tiles)


In [862]:
current_state

State(taken={2, 11, 12}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24})

# Solution with h3

In [863]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f3(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f3(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 412 steps (3 tiles)


In [864]:
current_state

State(taken={19, 13, 15}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 20, 21, 22, 23, 24})